In [1]:
import fitz
import re

In [2]:
doc = fitz.open(r'C:\Users\marcu\Python_Scripts\H332 X-PRESS MULHACEN 11.1.2022.pdf')

In [3]:
shipper = (33, 140)
unit = (222, 290)
goods = (312, 445)
netw = (468, 502)
tare = (510, 547)
pod = (460, 550)
base_triangle = (33, 100, 547, 580) # första rektangeln på första bladet, mellan "Shipper :" och "Printed :". Lägger sedan till på y0 och y1

In [11]:
total_starts = []   # lista över rektanglar för "Shipper :" på alla blad
total_stops = []    # lista över rektanglar för "Units :" på alla blad
total_height = 0.0  # loop över summan av höjderna på alla blad
total_words = []    # lista över alla dokumentets ord på alla blad
rect_pods = []     # lista över rektanglar för "Port of Discharge" på alla blad
words_in_rectangles = [] # lista för ord inom rektanglarna, utan "headers"
total_pods = [] # lista över rektanglar och namn för PODs

for page in doc:

    # starts
    starts = page.search_for("Shipper :")
    for rect in starts:
        total_starts.append(rect + (0, total_height, 0, total_height)) # lägger till på höjden för y0 och y1

    # stops
    stops = page.searchFor("Units :")
    for rect in stops:
        total_stops.append(rect + (0, total_height, 0, total_height))

    # total_words
    words_in_page = page.get_text("words")
    for words in words_in_page:
        total_words.append([words[0], words[1] + total_height, words[2], words[3] + total_height, words[4]])

    # rect_pods
    pods = page.searchFor('Port of discharge')
    for rect in pods:      
        rect_pods.append(rect + (0, total_height + 12.20998, 0, total_height + 16.08598)) # lägger till på y0 och y1 för att hitta POD
    
    # words_in_rectangles
    base_triangle = (33, 100 + total_height, 547, 580 + total_height) # Utgår från basrektangel (33, 100, 547, 580) och lägger på totalhöjden
    words_in_rectangles += [word for word in total_words if fitz.Rect(word[:4]).intersect(base_triangle)] # Tar fram alla ord inom rektangeln ovan

    total_height += page.rect.height # lägger till totalhöjden varje loop


#rect_pods 2

pod_names = [' '.join([words[4] for words in total_words if fitz.Rect(words[:4]).intersect(pod)]) for pod in rect_pods] # Tar fram alla POD is en lista

for rect, pod in zip(rect_pods, pod_names):      #lägger ihop rektanglarna med POD-namn
    total_pods.append([rect[0], rect[1], rect[2], rect[3], pod])





In [102]:
def gather_data(start, stop):
    rektangel = (464, start[1], 555, stop[3]) # rektangel för området
    samma_rad = start[1]        # värdet uppdateras varje loop med värdet från föregående loop
    lista_gathered_data = []    # varje ny rad representerar en instans i listan
    ordlista = ""
    
    for word in total_words:
            
        if fitz.Rect(word[:4]).intersects(rektangel):
            
            if samma_rad != word[1]:
                lista_gathered_data.append(ordlista.strip())
                ordlista = ""
                ordlista = word[4]

            elif samma_rad == word[1]:
                ordlista += " " + word[4]

            samma_rad = word[1]

    lista_gathered_data.append(ordlista.strip()) # lägger till den sista raden i loopen
       
    return lista_gathered_data



In [113]:
netw_rect = (468, 100, 502, 416)



def get_goods_info(goods, start, stop):

    rectangle = (goods[0], start[1], goods[1], stop[3])
    samma_rad = start
    lista_gathered_data = []
    lista_ord = ""

    for word in words_in_rectangles:

        if fitz.Rect(word[:4]).intersect(rectangle):
            
            wording = re.sub(r"'", "", word[4])                     # Tar bort ' från exv. "20' DC"

            if samma_rad == word[1]:                                # Om y-värdet på föregående rad i loop är samma som y-värdet för denna runda
                lista_ord += " " + wording

            elif samma_rad != word[1]:                              # Om y-värdet inte stämmer överens
                    lista_gathered_data.append(lista_ord.strip())
                    lista_ord = ""
                    lista_ord = wording

            samma_rad = word[1]                                     # Sparar Y-värdet för raden för denna loop. Används i nästa runda

    lista_gathered_data.remove('')                                  # Tar bort första "" i listan
    
    customs_found = [id for id, value in enumerate(lista_gathered_data) if re.search(r'^Customs', value)]  # Söker efter "Customs" i lista_gathered_data och letar match
    
    lista_ytterligare_info = lista_gathered_data[customs_found[0]:]                                         # Ny lista som kopierar info från match med "Customs" och till slutet
    lista_gathered_data = lista_gathered_data[:customs_found[0]]                                            # Gör om lista och tar info fram till "Customs" match

    return lista_gathered_data, lista_ytterligare_info

get_goods_info(goods, total_starts[20], total_stops[20])

(['20 DC', '3 PKG STAINLESS STEEL'],
 ['Customs status "N"',
  'Transhipment by MSC CATERINA',
  'Voy. NA202A',
  'ETD 23.1.2022 NAVEGANTES, SC',
  'ETA 12.2.2022',
  'Ref. 79096203 OPS',
  '44089/1632869',
  'FREIGHT PAYABLE AT As per C/P'])

In [96]:
new_list2 = ['Customs status "N"',
  'Transhipment by MSC PETRA',
  'Voy. NT202A',
  'ETD 25.1.2022 GEBZE ETA',
  '1.2.2022',
  'Ref. 11221059091 OPS',
  '43914/1631983',
  'FREIGHT PAYABLE AT As per C/P']

new_list3 = ['Customs status "N"',
  'Transhipment by MSC CATERINA',
  'Voy. NA202A',
  'ETD 23.1.2022 NAVEGANTES, SC',
  'ETA 12.2.2022',
  'Ref. 79096203 OPS',
  '44089/1632869',
  'FREIGHT PAYABLE AT As per C/P']


['Customs status "N"', 'Transhipment by MSC PETRA', 'Voy. NT202A', 'ETD 25.1.2022 GEBZE ETA', '1.2.2022', 'Ref. 11221059091 OPS', '43914/1631983', 'FREIGHT PAYABLE AT As per C/P']


In [123]:
def breakdown_ytterligare_info():

    ny_lista = new_list2

    customs_status = ""
    ocean_vessel = ""
    voyage = ""
    final_dest = ""

    for val in ny_lista:

        if re.search(r'^Customs\s.{6}\s\".\"', val, i):
            customs_status = val[16]

        if re.search(r'Transhipment\sby\s', val, i):
            ocean_vessel = val[16:]

        if re.search(r'Voy.', val, i):      # Ej klar
            voyage = val[4:]

        if re.search(r'', val, i):
            final_dest =

    return customs_status, ocean_vessel, voyage

breakdown_ytterligare_info()

('N', 'MSC PETRA', ' NT202A')

In [62]:
test_list = [1, 4, 5, 6, 4, 5, 6, 5, 4]

size = len(test_list)
idx_list = [idx + 1 for idx, val in
            enumerate(test_list) if val == 5]

new_list = []
for i,j in zip([0, 3, 6, 8], [3, 6, 8, 9]):
    new_list.append(test_list[i: j])

print(new_list)


[[1, 4, 5], [6, 4, 5], [6, 5], [4]]
